In [ ]:
!pip install openpyxl

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("df_train.xlsx", sheet_name="Sheet1")

labels = {
    "Нет": 0,
    "12.16. часть 1": 1,
    "12.17": 2,
    "12.12": 3,
    "12.15": 4,
    "12.16 часть 2": 4
}

def apply_label(text):
    for key, value in labels.items():
        if key in text:
            return value
    return 0 

df.drop(columns=["сумма штрафа, руб."], inplace=True)
df['label'] = df['наименование нарушения'].apply(apply_label)
df.drop(columns=['наименование нарушения'], inplace=True)
df.rename(columns={"время нарушения (в секундах)": "sec", "номер видео": "file"}, inplace=True)

df.head()

,file,sec,label
0,akn00006_fqGg6dtL,18,1
1,akn00006_fqGg6dtL,75,4
2,akn00006_fqGg6dtL,249,1
3,akn00006_fqGg6dtL,250,2
4,akn00006_fqGg6dtL,263,1


In [ ]:
!pip install onnxruntime-silicon

In [3]:
import onnxruntime as ort
import onnx

model = onnx.load("models/sign/sign_yolo.onnx")
model.ir_version = 9
onnx.save(model, "models/sign/sign_yolo.onnx")

model = onnx.load("models/traffic-lights/traffic-lights.onnx")
model.ir_version = 9
onnx.save(model, "models/traffic-lights/traffic-lights.onnx")

model = onnx.load("models/twin_lite/pretrained/twin_lite.onnx")
model.ir_version = 9
onnx.save(model, "models/twin_lite/pretrained/twin_lite.onnx")

# Инициализация ONNX моделей
# Степ, это нужно будет убрать
# Либо посмотреть оптимизации для куды, чтобы быстрее было
session_options = ort.SessionOptions()
session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
session_options.intra_op_num_threads = 1
session_options.inter_op_num_threads = 1

# Здесь убрать sess_options=session_options, providers=["CoreMLExecutionProvider"]
# Либо посмотреть оптимизации для куды, чтобы быстрее было
sign_model = ort.InferenceSession('models/sign/sign_yolo.onnx', sess_options=session_options, providers=["CoreMLExecutionProvider"])
traffic_lights_model = ort.InferenceSession('models/traffic-lights/traffic-lights.onnx', sess_options=session_options, providers=["CoreMLExecutionProvider"])
lane_model = ort.InferenceSession('models/twin_lite/pretrained/twin_lite.onnx', sess_options=session_options, providers=["CoreMLExecutionProvider"])

2024-11-10 06:35:57.647273 [W:onnxruntime:, graph.cc:108 MergeShapeInfo] Error merging shape info for output. '/model.22/Unsqueeze_16_output_0' source:{-1,-1} target:{-1,-1,1}. Falling back to lenient merge.
2024-11-10 06:35:57.647437 [W:onnxruntime:, graph.cc:108 MergeShapeInfo] Error merging shape info for output. '/model.22/Unsqueeze_15_output_0' source:{-1,-1} target:{-1,-1,1}. Falling back to lenient merge.
2024-11-10 06:35:57.647785 [W:onnxruntime:, graph.cc:108 MergeShapeInfo] Error merging shape info for output. '/model.22/Unsqueeze_11_output_0' source:{-1,-1} target:{-1,-1,1}. Falling back to lenient merge.
2024-11-10 06:35:57.647843 [W:onnxruntime:, graph.cc:108 MergeShapeInfo] Error merging shape info for output. '/model.22/Unsqueeze_10_output_0' source:{-1,-1} target:{-1,-1,1}. Falling back to lenient merge.
2024-11-10 06:35:57.648113 [W:onnxruntime:, graph.cc:108 MergeShapeInfo] Error merging shape info for output. '/model.22/Unsqueeze_6_output_0' source:{-1,-1} target:{-1

In [5]:
import cv2

def get_video_length_in_seconds(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Не удалось открыть видеофайл.")

    # Общее количество кадров
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Частота кадров (FPS)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    # Длина видео в секундах
    video_length = frame_count / fps
    return video_length

In [6]:
import os
from ansamble_preprocessing import process_video
from collections import defaultdict

directory_path = "videos/"

labels_dict = defaultdict()

processed_files = dict()
files_and_folders = os.listdir(directory_path)

rename_files_in_table = {
    "akn00036": "akn00036_iXsAm22P.mov",
    "akn00048": "AKN00048.mp4",
    "akn00077": "AKN00077.mp4",
    "akn00078": "AKN00078.mp4",
    "akn00080": "akn00080_f9XXm8bV.mov",
    "akn00085": "AKN00085.mp4",
    "akn00047": "AKN00047.mp4",
    "akn00088": "AKN00088.mp4"
}

for id, row in df.iterrows():
    for file_name in files_and_folders:
        file_real = row['file']
        if row['file'] in rename_files_in_table:
            file_real = rename_files_in_table[row['file']]
        if file_real in file_name:
            video_path = os.path.join(directory_path, file_name)
            if file_name not in processed_files:
                processed_files[file_name] = process_video(video_path, sign_model, traffic_lights_model, lane_model)
                
            if row['file'] not in labels_dict:
                labels_dict[file_name] = [0] * int(get_video_length_in_seconds(video_path))
            labels_dict[file_name][row['sec']] = row['label']

In [7]:
labels_dict.keys()

dict_keys(['akn00006_fqGg6dtL.mov', 'akn00036_iXsAm22P.mov', 'AKN00048.mp4', 'AKN00077.mp4', 'AKN00078.mp4', 'akn00080_f9XXm8bV.mov', 'AKN00085.mp4', 'AKN00047.mp4', 'AKN00088.mp4'])

In [8]:
# Импорт необходимых библиотек
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from classification_model import LightTrafficViolationClassifier, train_model, CustomViolationLoss

device = "cuda" if torch.cuda.is_available() else "cpu"

class TrafficViolationDataset(Dataset):
    def __init__(self, processed_files, labels_dict):
        self.processed_files = processed_files
        self.labels_dict = labels_dict
        self.video_names = list(processed_files.keys())
        
    def __len__(self):
        return len(self.video_names)
    
    def __getitem__(self, idx):
        video_name = self.video_names[idx]
        frames = self.processed_files[video_name]  # цепочка обработанных кадров (размер [D, H, W, 3])
        labels = self.labels_dict[video_name]      # список лейблов (размер D)
        
        # Преобразуем размерности кадров [D, H, W, 3] -> [D, 3, H, W]
        frames_tensor = torch.tensor(frames, dtype=torch.float32).permute(0, 3, 1, 2)
        labels_tensor = torch.tensor(labels, dtype=torch.long)
        
        return frames_tensor, labels_tensor

num_classes = len(set([label for labels in labels_dict.values() for label in labels]))

# Подготовка данных и DataLoader
video_names = list(processed_files.keys())
train_videos, test_videos = train_test_split(video_names, test_size=0.2, random_state=42)
train_dataset = TrafficViolationDataset({v: processed_files[v] for v in train_videos}, {v: labels_dict[v] for v in train_videos})
val_dataset = TrafficViolationDataset({v: processed_files[v] for v in test_videos}, {v: labels_dict[v] for v in test_videos})

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Инициализация модели
# Получаем размеры входных данных из loader
sample_frames = next(iter(train_loader))[0]
_, _, _, height, width = sample_frames.shape

# Инициализация модели
num_classes = len(set([label for labels in labels_dict.values() for label in labels]))
model = LightTrafficViolationClassifier(
    input_height=height,
    input_width=width,
    num_classes=num_classes
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Определение функции потерь и оптимизатора
criterion = CustomViolationLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)

# Добавьте scheduling
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=1e-3,
    epochs=10,
    steps_per_epoch=len(train_loader),
    pct_start=0.3
)

# Обучение модели
train_model(
    model, 
    train_loader,
    criterion,
    optimizer,
    device,
    num_epochs=10,
    validation_loader=val_loader,
    window_size=5,
    save_path='models/classification_model/cls_model.pth',
    scheduler=scheduler
)

/var/folders/6v/rd9mywh12msdrg5xk2zvrs480000gn/T/ipykernel_90771/1733897788.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  frames_tensor = torch.tensor(frames, dtype=torch.float32).permute(0, 3, 1, 2)


: 

In [ ]:
import numpy as np
import pandas as pd

# Адаптированная функция для подсчета метрики с числовыми метками
def pre_calc_score_numeric(gt, sub):
    pred_seconds = []
    correct_predictions = []
    AE_count_rules_FP = []
    AE_count_rules_FN = []

    # Проходим по каждому нарушению из gt
    for i, r_gt in gt.iterrows():
        # Отбираем строки для конкретного видео и нарушения в сабмишене
        video_sub = sub[(sub['номер видео'] == r_gt['номер видео']) & (sub['код нарушения'] == r_gt['код нарушения'])]
        video_gt = gt[(gt['номер видео'] == r_gt['номер видео']) & (gt['код нарушения'] == r_gt['код нарушения'])]

        # Если нарушения в предсказаниях отсутствуют
        if len(video_sub) == 0:
            pred_seconds.append(np.NaN)
            correct_predictions.append(False)
            FP = max(0 - len(video_gt), 0) # False Positive
            FN = abs(min(0 - len(video_gt), 0)) # False Negative
            AE_count_rules_FP.append(FP)
            AE_count_rules_FN.append(FN)
            continue

        # Время реального нарушения
        true_sec = r_gt['время нарушения (в секундах)']

        # Находим ближайшую предсказанную секунду
        pred_sec = min(video_sub['время нарушения (в секундах)'].values, key=lambda x: abs(x - true_sec))
        pred_seconds.append(pred_sec)

        # Проверяем попадание в 5-секундный интервал
        correct_prediction = np.abs(pred_sec - true_sec) < 5
        correct_predictions.append(correct_prediction)

        # Подсчитываем FP и FN
        FP = max(len(video_sub) - len(video_gt), 0)
        FN = abs(min(len(video_sub) - len(video_gt), 0))
        AE_count_rules_FP.append(FP)
        AE_count_rules_FN.append(FN)

    # Добавляем результаты в таблицу gt
    gt['pred_seconds'] = pred_seconds
    gt['Корректность предсказания'] = correct_predictions
    gt['Лишние нарушения (FP)'] = AE_count_rules_FP
    gt['Недостающие нарушения (FN)'] = AE_count_rules_FN
    return gt

# Функция для вычисления итоговой метрики
def calculate_score(result_table):
    return max(0, (result_table['Корректность предсказания'].sum() - (result_table['Лишние нарушения (FP)'].sum() * 0.5)) / len(result_table))